# Predykcja zużycia energii elektrycznej z wykorzystaniem sieci LSTM

## 1. Import bibliotek i załadowanie danych

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Sprawdzenie, czy GPU jest dostępne
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Załadowanie przykładowego zbioru danych (link można zamienić na inny publiczny zbiór)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/household_power_consumption.zip"
df = pd.read_csv(url, sep=';', parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='datetime')

# Wyświetlenie pierwszych wierszy danych
df.head()
    

## 2. Wstępne przetwarzanie danych

In [ ]:

# Użycie jednej kolumny: Global_active_power
data = df['Global_active_power'].dropna()

# Skalowanie danych do zakresu [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.values.reshape(-1, 1))

# Konwersja do szeregów czasowych: sekwencje X -> y
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

SEQ_LENGTH = 48  # sekwencja 48 próbek (np. 48 godzin)
x, y = create_sequences(data_scaled, SEQ_LENGTH)

# Podział na zbiory treningowe i testowe
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Konwersja do tensora PyTorch
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
    

## 3. Definicja modelu LSTM

In [ ]:

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Ostatnie wyjście LSTM
        return out

# Parametry modelu
input_dim = 1
hidden_dim = 50
num_layers = 2
output_dim = 1

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim).to(device)
    

## 4. Trening modelu

In [ ]:

# Funkcja straty i optymalizator
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Przygotowanie danych do DataLoadera
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Trening
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")
    

## 5. Ewaluacja modelu

In [ ]:

# Prognozowanie na zbiorze testowym
model.eval()
with torch.no_grad():
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    predictions = model(x_test).cpu().numpy()

# Odwrotne skalowanie wyników
y_test = scaler.inverse_transform(y_test.cpu().numpy().reshape(-1, 1))
predictions = scaler.inverse_transform(predictions)

# Wizualizacja wyników
plt.figure(figsize=(10, 6))
plt.plot(y_test[:100], label="Rzeczywiste", color="blue")
plt.plot(predictions[:100], label="Prognozy", color="red")
plt.legend()
plt.title("Porównanie prognoz z rzeczywistymi danymi")
plt.show()
    

## 6. Podsumowanie

Model LSTM został zastosowany do predykcji zużycia energii elektrycznej. Wyniki pokazują, że model jest w stanie uchwycić trend w danych szeregów czasowych, choć mogą wystąpić pewne odchylenia w prognozach. Możliwe dalsze kroki to optymalizacja hiperparametrów i analiza wpływu cech dodatkowych, takich jak dane pogodowe.